In [5]:
#getting weird error only on my computer. 
import pandas as pd
import numpy as np
import glob
import matplotlib as mpl
import matplotlib.pyplot as plt 
import os
import more_itertools as locate
import re
####################################################
#USER INPUTS HERE!!!!!

#############FilePaths
FilePathList="/Users/timmo/Documents/RAPFLAB/code/blanktest" #where is the mass list located
FilePathBlank="/Users/timmo/Documents/RAPFLAB/code/blanktest/Processed" #where you have your blank files
SaveLocation="/Users/timmo/Documents/RAPFLAB/code/blanktest/processed" #where you want to save your processed blank files
FilePathData="/Users/timmo/Documents/RAPFLAB/code/MS" #where your data is stored
SaveLocation2="/Users/timmo/Documents/RAPFLAB/code/Processed" #where you save your final processed files


#############FileNames
ListFile="IncompCombinedmassList.csv" #Mass list file name 
BlankFile="stdev_blank.csv" #filename of blank file with its stdevs. Should stay constant
FileName="_Processed.csv" #filename 

###########Small stuff
FileType="csv" #sets filetype
ErrorRange = 0.00005 #error range. Will be +- to get range and see if values fit in range. 25 ppm error range     
###################################################






####################################### List initialization
MassNumbersList=[] #creates an empty list which will be filled by the numbers in the mass list file
os. chdir(FilePathList) #sets working directory to FilePathList
CurrFile = pd.read_csv(ListFile) #reads the file masslist.csv from the filepath --> since we only want to read one file was easier to just specify file
MassNumberNames=[]
mzlistRep=[]
TotalList=[]
TotalList2=[]
mzlistRepTot=[]
FinalList=[]
#######################################






#####################################PART 1 of CODE
#Gets standard deviation for set blank files and puts them in a processed stdev file


for i in (range(len(CurrFile))):
    MassNumbersList.append(CurrFile['mz_list'][i]) #appends value number i from masslist.csv in the column mz_list
    MassNumberNames.append(CurrFile['Species'][i])#appends species i rom masslist.csv in the column Species
    #adds species and mz_list from the current file to the currfile list

os. chdir(FilePathBlank) #updates file path to the folder with actual files for processing
files = sorted(glob.glob("*"+"Blank"+"*"+FileType)) #gets all files of chosen filetype
num_files = len(files) #how many files are you reading in

##############################Main loop
for entry in range(num_files): #goes through all files
    name = files[entry] 
    CurrFile = pd.read_csv(files[entry],skiprows=0) #reads files
    FullSplit = name.split(".") #gets all parts of the file name
    NameList=(str(FullSplit[0]))
    mzlistRep=[]
    ProcessedX=[] #values for X(Thompsons) will be saved here
    ProcessedY=[] #values for Y(Counts) will be saved here
    PPM=[] #PPM values will be saved here
    names=[]
    #gets the third part of the name (0,1,[2]). Adds it to the name list
    for mass in range(len(MassNumbersList)): #goes through each mast in the MassesList
        MassHigh=((MassNumbersList[mass])+(MassNumbersList[mass]*ErrorRange)) #upper range of Mass
        MassLow=((MassNumbersList[mass])-(MassNumbersList[mass]*ErrorRange)) #lower range of Mass
        fixed=CurrFile.drop(columns=["#Point"])
        fit=fixed["X(Thompsons)"].between(MassLow, MassHigh, inclusive = True) #checks if value for X(Thompson) fits between mass low and mass high
        ####Part two of function
        for i in range(len(fit)): #checks if values in fit are true or false
             if fit[i]== True: #if True then appends values
                    ProcessedX.append(fixed["X(Thompsons)"][i])
                    ProcessedY.append(fixed["Y(Counts)"][i])
                    PPM.append(((abs((fixed["X(Thompsons)"][i])-MassNumbersList[mass]))/MassNumbersList[mass])*1000000)
                    mzlistRep.append(MassNumbersList[mass])
                    names.append(MassNumberNames[mass])
             
        if sum(fit) == False: #if all values are false and nothing fits then will append a 0. This makes sure you get the correct
                                  # number of values no matter what
                ProcessedX.append(0)  
                ProcessedY.append(0) #appends 0 for both 
                PPM.append(0)
                mzlistRep.append(MassNumbersList[mass])
                names.append(MassNumberNames[mass])      
                
    
    for i in range(len(ProcessedX)): 
        TotalList.append(ProcessedX[i])
        TotalList2.append(ProcessedY[i])
        mzlistRepTot.append(mzlistRep[i])
        #These list are a total of all values. The massnumber and counts will match those in the masslist. This will let you pick out all values of a certain mass from the masslist and get matching counts and X(Thompsons)
            
for mass in range(len(MassNumbersList)):
    Matchlist=[]
    List=[]
    
    ###Finds all values of the masslist where it is equal to Massnumberslist[mass]. Pulls all values and puts them into a list. Makes an array with values for matching mass list
    searchval = (MassNumbersList[mass])
    ii = np.where(mzlistRepTot == searchval)[0] #goes through and adds values that match to ii list
    List=str(ii).split(" ") 
    str(List).strip()
    res = []
    for i in List:
        m = re.search(r"\b(\d{1,2})\b", i)
        if m:
            res.append(m.group())
    ###Code I found online and edited a bit
    
    ### Makes new list that adds counts for matching part of mass list
    Final=[]
    for i in range(len(res)):
        Final.append((TotalList2[int(res[i])]))
    ###  
    
    
    ####calculates standard deviation for a certain mass
    stdList=[]
    for i in range(len(Final)):
        mean = sum(Final) / len(Final)
        variance = sum([((x - mean) ** 2) for x in Final]) / len(Final)
        std = variance ** 0.5
    stdList.append(std)
    for i in range(len(stdList)):    
        FinalList.append((stdList[i]))
    #appends all standard deviations to std list
        

        
IndFile = pd.DataFrame(columns = ["mz_list","standard deviation"])
IndFile["mz_list"]=MassNumbersList
IndFile["standard deviation"]=FinalList
IndFile.to_csv(SaveLocation+BlankName)
#makes stdev file by adding mzlist and having a standard deviation for each mass

########################################################################
########################################################################
#PART TWO OF THE PROGRAM. Part 1 does stdev stuff. Lots of repeating code here
#Works in a similar way, but just implements code to make a file with PPM, detected, counts, thompsons etc for actual data
#some lists need to stay down here to re-initialize, so I didnt change much.
########################################################################
########################################################################




#initializing some random stuff
os. chdir(FilePathList) #sets working directory to FilePathList
MassNumbersList=[] #creates an empty list which will be filled by the numbers in the mass list file
MassNumberNames=[]
#######################################
CurrFile = pd.read_csv(ListFile) #reads the file masslist.csv from the filepath --> since we only want to rea

for i in (range(len(CurrFile))):
    MassNumbersList.append(CurrFile['mz_list'][i]) #appends value number i from masslist.csv in the colum mz_list
    MassNumberNames.append(CurrFile['Species'][i])

os. chdir(SaveLocation)
Blanks=[]#list for blank standard deviations
CurrFile = pd.read_csv(BlankFile)

#Adds standard deviation for each mass
for i in (range(len(CurrFile))):
    Blanks.append(CurrFile['standard deviation'][i])

    
os. chdir(FilePathData) #updates file path to the folder with actual files for processing
files = sorted(glob.glob("*"+FileType)) #gets all files of chosen filetype
num_files = len(files) #how many files are you reading in
##############################

ErrorRange = 0.00005 #error range. Will be +- to get range and see if values fit in range. 25 ppm error range     
mzlistRep=[]



for entry in range(num_files): #goes through all files
    
    
    #################Reinitializes the lists that we need to reinitialize. Not all of them need to be reinitialized. 
    name = files[entry] 
    CurrFile = pd.read_csv(files[entry],skiprows=1) #reads files
    FullSplit = name.split(".") #gets all parts of the file name
    NameList=(str(FullSplit[0]))
    mzlistRep=[]
    ProcessedX=[] #values for X(Thompsons) will be saved here
    ProcessedY=[] #values for Y(Counts) will be saved here
    PPM=[] #PPM values will be saved here
    names=[]
    BlankRep=[]
    Detected=[]
    
    #####same basic loop. Gets values of the file that correspond to matching massnumberslist 
    for mass in range(len(MassNumbersList)): #goes through each mast in the MassesList
        MassHigh=((MassNumbersList[mass])+(MassNumbersList[mass]*ErrorRange)) #upper range of Mass
        MassLow=((MassNumbersList[mass])-(MassNumbersList[mass]*ErrorRange)) #lower range of Mass
        fixed=CurrFile.drop(columns=["#Point"]) #drops point column
        fit=fixed["X(Thompsons)"].between(MassLow, MassHigh, inclusive = True) #checks if value for X(Thompson) fits between mass low and mass high
        ####Part two of function
        for i in range(len(fit)): #checks if values in fit are true or false
             if fit[i]== True: #if True then appends values
                    ProcessedX.append(fixed["X(Thompsons)"][i])
                    ProcessedY.append(fixed["Y(Counts)"][i])
                    PPM.append(((abs((fixed["X(Thompsons)"][i])-MassNumbersList[mass]))/MassNumbersList[mass])*1000000)
                    mzlistRep.append(MassNumbersList[mass]) #adds matching mzlistRep
                    names.append(MassNumberNames[mass]) #adds matching name
                    BlankRep.append(Blanks[mass]) #only really different part. Adds standard dev of blanks to a list

                  
            
        if sum(fit) == False: #if all values are false and nothing fits then will append a 0. This makes sure you get the correct
                                  # number of values no matter what
                ProcessedX.append(0)  
                ProcessedY.append(0) #appends 0 for both 
                PPM.append(0)
                mzlistRep.append(MassNumbersList[mass])
                names.append(MassNumberNames[mass])  
                BlankRep.append(Blanks[mass])#adds standard dev. This will not really be necessary, but helps for the final doc.
                #same as first, except when nothing matches
                
######This code is mostly the same as the first repetition of the code.                
                
    ######Math for if it is detected. Currently its just 3x the stdev of the blank. If counts is lower, then it will say not detected. Otherwise it says detected            
    for i in range(len(ProcessedY)):
        if ProcessedY[i]>=(3*(BlankRep[i])):
              Detected.append("Detected")
        elif ProcessedY[i]<(3*(BlankRep[i])):
               Detected.append("Not Detected")
        else:
            continue

    #Makes the actual file
    IndFile = pd.DataFrame(columns = ["Compound Names","mz_list",NameList+"_mass",NameList+"_count",NameList+"_PPM", "Not Detected"]) #columns mz_list, mass, and count +NameList[i]
    IndFile["Compound Names"]=names
    IndFile["mz_list"]=mzlistRep
    IndFile[NameList+"_mass"]=ProcessedX
    IndFile[NameList+"_count"]=ProcessedY #for this and the line before, it reads the values of processed from i*length
    IndFile[NameList+"_PPM"]=PPM
    IndFile["Blank STDev"]=BlankRep
    IndFile["Not Detected"]=Detected
    IndFile.to_csv(SaveLocation2+NameList+FileName)   


Empty DataFrame
Columns: []
Index: []


KeyError: "['#Point'] not found in axis"